# EEG Motor Movement/Imagery Dataset

Published: Sept. 9, 2009. Version: 1.0.0

https://physionet.org/content/eegmmidb/1.0.0/

In [1]:
from ica_benchmark.io.load import Physionet_2009_Dataset, BCI_IV_Comp_Dataset, OpenBMI_Dataset
from pathlib import Path
import mne

physionet_dataset_folderpath = Path('/home/paulo/Documents/datasets/Physionet')
bci_dataset_folderpath = Path('/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/')
bci_test_dataset_folderpath = Path('/home/paulo/Documents/datasets/BCI_Comp_IV_2a/true_labels/')
openbmi_dataset_folderpath = Path('/home/paulo/Documents/datasets/OpenBMI/edf/')

physionet_dataset = Physionet_2009_Dataset(physionet_dataset_folderpath)
bci_dataset = BCI_IV_Comp_Dataset(bci_dataset_folderpath, test_folder=bci_test_dataset_folderpath)
openbmi_dataset = OpenBMI_Dataset(openbmi_dataset_folderpath)

display(physionet_dataset.list_subject_filepaths())
display(bci_dataset.list_subject_filepaths())
display(openbmi_dataset.list_subject_filepaths())

,path,uid,trial,task
0,/home/paulo/Documents/datasets/Physionet/files...,92,9,3
1,/home/paulo/Documents/datasets/Physionet/files...,92,1,10
2,/home/paulo/Documents/datasets/Physionet/files...,92,5,3
3,/home/paulo/Documents/datasets/Physionet/files...,92,4,2
4,/home/paulo/Documents/datasets/Physionet/files...,92,2,11
...,...,...,...,...
1521,/home/paulo/Documents/datasets/Physionet/files...,28,14,4
1522,/home/paulo/Documents/datasets/Physionet/files...,28,8,2
1523,/home/paulo/Documents/datasets/Physionet/files...,28,4,2
1524,/home/paulo/Documents/datasets/Physionet/files...,28,5,3


,path,train,uid
0,/home/paulo/Documents/datasets/BCI_Comp_IV_2a/...,False,7
1,/home/paulo/Documents/datasets/BCI_Comp_IV_2a/...,False,3
2,/home/paulo/Documents/datasets/BCI_Comp_IV_2a/...,False,8
3,/home/paulo/Documents/datasets/BCI_Comp_IV_2a/...,True,8
4,/home/paulo/Documents/datasets/BCI_Comp_IV_2a/...,True,3
5,/home/paulo/Documents/datasets/BCI_Comp_IV_2a/...,True,4
6,/home/paulo/Documents/datasets/BCI_Comp_IV_2a/...,True,7
7,/home/paulo/Documents/datasets/BCI_Comp_IV_2a/...,False,4
8,/home/paulo/Documents/datasets/BCI_Comp_IV_2a/...,False,1
9,/home/paulo/Documents/datasets/BCI_Comp_IV_2a/...,True,2


,path,train,session,uid
0,/home/paulo/Documents/datasets/OpenBMI/edf/ses...,True,2,25
1,/home/paulo/Documents/datasets/OpenBMI/edf/ses...,True,2,15
2,/home/paulo/Documents/datasets/OpenBMI/edf/ses...,True,2,41
3,/home/paulo/Documents/datasets/OpenBMI/edf/ses...,False,2,12
4,/home/paulo/Documents/datasets/OpenBMI/edf/ses...,False,2,37
...,...,...,...,...
211,/home/paulo/Documents/datasets/OpenBMI/edf/ses...,False,1,48
212,/home/paulo/Documents/datasets/OpenBMI/edf/ses...,False,1,14
213,/home/paulo/Documents/datasets/OpenBMI/edf/ses...,True,1,16
214,/home/paulo/Documents/datasets/OpenBMI/edf/ses...,True,1,48


In [2]:
mne.set_log_level(False)

kwargs = dict(
    reject=False,
    tmin=0,
    tmax=1,
)
epochs = [
    physionet_dataset.load_subject("1", **kwargs),
    bci_dataset.load_subject("1", **kwargs),
    openbmi_dataset.load_subject("1", session=1, **kwargs)
]

/home/paulo/Documents/GIT/BCI_MsC/ica_benchmark/io/load.py:458: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = concatenate_epochs(epochs_list)
/home/paulo/anaconda3/envs/bci_update/lib/python3.9/site-packages/mne/io/edf/edf.py:1123: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/home/paulo/anaconda3/envs/bci_update/lib/python3.9/contextlib.py:124: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
/home/paulo/anaconda3/envs/bci_update/lib/python3.9/site-packages/mne/io/edf/edf.py:1123: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/home/paulo/anaconda3